In [30]:
# import os
# os.environ['OPENDP_LIB_DIR'] = '/Users/ethancowan/IdeaProjects/openDP/rust/target/release'

from opendp.trans import *
from opendp.meas import *
from opendp.core import *

enable_features("floating-point")

# establish data information
data_path = os.path.join('.', 'data', 'PUMS_california_demographics_1000', 'data.csv')
var_names = ["age", "sex", "educ", "race", "income", "married"]

# TODO: Remove column headers
with open(data_path) as input_data:
    data = input_data.read()

In [31]:
def _binary_search(predicate, start, end):
    if start > end:
        raise ValueError

    if not predicate(end):
        raise ValueError("no possible value in range")

    while True:
        mid = (start + end) / 2
        passes = predicate(mid)

        if passes and end - start < .00001:
            return mid

        if passes:
            end = mid
        else:
            start = mid

def check_scale(scale, preprocessor, dataset_distance, epsilon):
    """
    Return T/F
    :param scale:
    :param preprocessor:
    :param dataset_distance:
    :param epsilon:
    :return:
    """
    return (preprocessor >> make_base_laplace(scale)).check(dataset_distance, epsilon)


In [32]:
def make_dp_mean(col_names, index, lower, upper, n, epsilon):
    """
    Draft of a function to be used on the backend for DPCreator
    :param index: Column index to select data from
    :param data_type: Type to cast data to
    :param lower: Lower bound for clamp
    :param upper: Upper bound for clamp
    :param n: Estimated number of values in data
    :param epsilon: Privacy budget
    :return:
    """
    preprocessor = (
        # Convert data into Vec<Vec<String>>
        make_split_dataframe(separator=",", col_names=col_names) >>
        # Selects a column of df, Vec<str>
        make_select_column(key=index, T=str) >>
        # Cast the column as Vec<Optional<Float>>
        make_cast(TI=str, TO=float) >>
        # Impute missing values to 0 Vec<Float>
        make_impute_constant(0.) >>
        # Clamp age values
        make_clamp(lower, upper) >>
        make_resize_bounded(0., n, lower, upper) >>
        make_bounded_mean(lower, upper, n=n, T=float)
    )
    scale = _binary_search(lambda s: check_scale(s, preprocessor, 1, epsilon), 0., 10.)
    preprocessor = preprocessor >> make_base_laplace(scale)
    return preprocessor(data)



In [33]:
epsilon = 1.
col_names = ["A","B","C","D","E"]
column = "E"
res = make_dp_mean(col_names, column, 0., 10000., 1000, epsilon)

print(f"Epsilon: {epsilon}\nColumn: {column}\nDP Mean: {res}")


Epsilon: 1.0
Column: E
DP Mean: 7827.314003837994
